In [1]:
# cd C:\Users\liamf\OneDrive\Desktop\Uni\Document_Processing\2022S1\assignments\assignment-3-start
# python -m http.server 8080
import requests
from bs4 import BeautifulSoup
import spacy
html = requests.get(url = 'http://localhost:8080/student.html').text
htmlParse = BeautifulSoup(html, 'html.parser').find_all('p')

text = ' '.join(x.string for x in htmlParse)


## QUESTION 2

In [119]:
def get_entities(sent):
  subj = ""           # Variable for storing the subject.
  obj = ""

  prv_tok_dep = ""    # Variable for dependency tag of previous token in the sentence. 
  prv_tok_text = ""   # Variable for previous token in the sentence.

  prefix = ""         # Variable for storing compounds.
  modifier = ""       # Variable for storing modifieres.

  # Loop through the tokens in the sentence.   
  for idt, tok in enumerate(sent):
    # Check if a token is a punctuation mark or not.
    if tok.dep_ != "punct":
      if tok.dep_ == 'attr':
          if(idt == len(sent) - 1):
                prefix = prv_tok_text + " " + tok.text
      # Check if a token is a compound one or not.
      if tok.dep_ == "compound":
        # If yes, then store the token in the prefix variable.
        prefix = tok.text
        # Check if the previous token was also a compound one.
        if prv_tok_dep == "compound":
          # If yes, then update the prefix variable.
          prefix = prv_tok_text + " "+ tok.text
      
      # Check if a token is a modifier or not.
      if tok.dep_.endswith("mod") == True:
        # If yes, then store the token in the modifier varible.
        modifier = tok.text
         # Check if the previous token was a compound one.
        if prv_tok_dep == "compound":
        # If yes, then update the modifier variable.  
          modifier = prv_tok_text + " "+ tok.text
    
      # Check if a token is the subject.
      if tok.dep_.find("subj") == True:
        # If yes, then concatenate the modifier, prefix, and token
        # and assign the result to the subject variable (ent1).
        subj = modifier +" "+ prefix + " "+ tok.text
        # Reset the following variables: prefix, modifier, prv_tok_dep, and prv_tok_text.
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      # Check if a token is the object.
      if tok.dep_.find("obj") == True:
        # If yes, then concatenate the modifier, prefix, and token 
        # and assign the result to the object variable (ent2).    
        obj = modifier +" "+ prefix +" "+ tok.text
        
      # Update the variable for the dependency tag for the previous token. 
      prv_tok_dep = tok.dep_
      # Update the variable for the previous token in the sentence.  
      prv_tok_text = tok.text


 
  return [subj.strip(), obj.strip()]

In [87]:
def getpred(sent):
    sentance = []
    for x in sent:
        if(x.dep_ != 'punct'):
            sentance.append(x)
       
    if(sentance[len(sentance) -1].dep_ == "attr"):
        sentance = sentance[:len(sentance) - 1]
    for idx, token in enumerate(sentance):
       
        if(idx < len(sentance) - 1):
            
            if(token.dep_ == 'det'):
                if(sentance[idx+1].dep_ == 'attr' or sentance[idx+1].dep_ == 'compound'):
                    sentance.remove(token)
                    continue
        if(token.dep_ == 'prep'):
            sentance = sentance[:idx + 1]
            break
            
    
    should_restart = True
    while(should_restart):
        for token in sentance:
            if(token.dep_ == 'aux' or token.dep_ == 'auxpass' or token.dep_ == 'ROOT'):
                should_restart = False
                break
            else:
                sentance.remove(token)
                should_restart = True
                break

    if(sentance[len(sentance) -1].dep_ == "nummod"):
        sentance = sentance[:len(sentance) - 1]
    return sentance

In [112]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
phrase_matcher = PhraseMatcher(nlp.vocab)

model = nlp(text)
sentances = []


for s in model.sents:
    sentances.append(s)

## splitting up sentnances with a 'cc' or joining word 
for sidx, sentance in enumerate(sentances):
    for idx, token in enumerate(sentance):
        if(token.dep_ == 'cc'):
            newsentance1 = sentance[idx:]
            newsentance2 = sentance[:idx]

            for newtoken in newsentance1:
                if(newtoken.dep_ == 'pobj'):
                    storetoken = newtoken
            
            newsentance1 = newsentance2

            newstring = ""
            for ridx, replacetoken in enumerate(newsentance1):
                if(replacetoken.dep_ == 'pobj'):
                    newstring += storetoken.text
                else:
                    newstring += replacetoken.text
                newstring += " "
         
            newsentance3 = nlp(newstring)
            sentances.remove(sentance)
            sentances.insert(sidx, newsentance3)
            sentances.insert(sidx, newsentance1)


[Kevin Walker is a student., Kevin Walker was born on 2001/07/24., Kevin Walker lives in Epping., Kevin Walker works at ALDI., Kevin is a friend of Alice Miller., Kevin is studying at Macquarie University., He has the student number 40048822., He is enrolled in COMP3100, He is enrolled in COMP3220 , Alice Miller is an alumna of Macquarie University., She is a friend of Kevin Walker.]


In [118]:
for current in sentances:
    print(get_entities(current))

['Kevin Walker', '']
['Kevin Walker', '2001/07/24']
['Kevin Walker', 'Epping']
['Kevin Walker', 'ALDI']
['Kevin', 'Alice Miller']
['Kevin', 'Macquarie University']
['He', 'student number']
['He', 'COMP3100']
['He', 'COMP3220']
['Alice Miller', 'Macquarie University']
['She', 'Kevin Walker']


In [116]:
current = sentances[0]
print(get_entities(current))
for x in current:
         print(x,'->', x.dep_)

['Kevin Walker', '']
Kevin -> compound
Walker -> nsubj
is -> ROOT
a -> det
student -> attr
. -> punct


In [209]:
import pandas as pd
source = ["Kevin Walker", "Kevin Walker","Kevin Walker","Kevin Walker","Kevin Walker","Kevin Walker","Kevin Walker","Kevin Walker","Kevin Walker","Alice Miller", "Alice Miller"]
edge = ["is a", "born on", "lives in", "works at", "is friend of", "is studying at", "has student number", "is enrolled in1", "is enrolled in2", "is alumna of", "is friend of"]
target = ["student", "2001-07-24", "Epping", "ALDI", "Alice Miller", "Macquarie University", "40048822", "COMP3100", "COMP3220", "Macquarie University", "Kevin Walker"]
d = {'name': source, 'edge': edge, 'target' : target}
# combine = zip(edge, target)
# d = dict(combine)
data = pd.DataFrame(data = d)
#data.to_csv("schemas.csv")
# peopleArray = data['source'].unique()
# peopleData = {'name' : peopleArray}
# peopleDF = pd.DataFrame(peopleData)
# peopleDF.to_csv("people.csv")



data = pd.DataFrame(columns = edge)



for idx, element in enumerate(edge):
    data.at[source[idx], edge[idx]] = target[idx] 


data.index.name = 'name'
data
data.to_csv("data.csv")

# QUESTION 3

In [212]:
import os
os.system("python -m rdfizer -c ./config.ini")

0